In [1]:
### Review Reasoning Capability (But now extra context is given via Bing API) https://github.com/microsoft/bing-search-sdk-for-python

### if ollama is not running put following line in terminal >> ollama serve 

In [2]:
import ollama
from pprint import pprint
import re, os 
from dotenv import load_dotenv
load_dotenv()


from news_search_client import NewsSearchClient
from azure.core.credentials import AzureKeyCredential
SUBSCRIPTION_KEY = os.getenv("BING_SEARCH_SUBSCRIPTION_KEY")
ENDPOINT = "https://api.bing.microsoft.com" + "/v7.0/"

def news_search(param_query):
    """NewsSearch.
    """
    client = NewsSearchClient(
        endpoint=ENDPOINT, credential=AzureKeyCredential(SUBSCRIPTION_KEY)
    )

    try:
        news_result = client.news.search(
            query=param_query, market="en-us", count=4, freshness="Week",
        )
        if news_result.value:
            return [{"role": "assistant", "content": n.description}  for n in news_result.value]

    except Exception as err:
        return []



In [3]:
news_search("Nissan Motor Co. merger")

[{'role': 'assistant',
  'content': 'Even if the merger is off, Nissan remains keen on working with Honda in areas such as electric vehicles and software defined vehicles.'},
 {'role': 'assistant',
  'content': 'Two of Japan’s largest automakers have been in talks to combine their operations to form one of the world’s biggest car groups.'},
 {'role': 'assistant',
  'content': 'The Wall Street Journal reported that Nissan was likely to walk away from a merger proposed in December that would create the world’s third-largest auto maker. It was a'},
 {'role': 'assistant',
  'content': 'Shares of Nissan Motor Co. and Honda Motor Co. rose following reports that the automakers’ proposed merger may be scrapped.'}]

In [4]:
news_search("TVS Holdings Ltd acquires  Home Credit India")

[{'role': 'assistant',
  'content': "The transaction was funded through a combination of the proceeds from the recent sale of TVS Holdings' real estate assets and borrowings from capital markets."},
 {'role': 'assistant',
  'content': 'TVS Holdings acquires 80.74% stake in Home Credit India for ₹554 crore, aiming to strengthen financial services sector.'},
 {'role': 'assistant',
  'content': 'Following the deal, together with TVS Credit, the Group is eyeing a lending book size of Rs 50,000 crore in the next three years'},
 {'role': 'assistant',
  'content': 'TVS Holdings acquires 80.74% stake in Home Credit India for ₹554 crore, strengthening its financial services sector presence.'}]

In [5]:

def only_return_main_answer(response_content_from_llm):
    """ This function removes the reasoning part of the response and only returns the main answer """
    tmp_answer = re.sub(r"<think>.*?</think>", "", response_content_from_llm, flags=re.DOTALL).strip()
    return(tmp_answer)


INSTRUCTION_PROMPT = "You are expert on M&A. Can you provide with a very brief summary of the following news article?"

def get_all_responses(news_article, news_topic):

    tmp_content = news_search(news_article)
    tmp_array = []
    tmp_array.append({"role": "system", "content": INSTRUCTION_PROMPT})
    for item in tmp_content:
        tmp_array.append(item)
    tmp_array.append({"role": "user", "content": news_article})

    response = ollama.chat(
        model="deepseek-r1:1.5b",
        messages=tmp_array
    )
    response_content_from_llm = response["message"]["content"]
    only_answer_no_reasoning = only_return_main_answer(response_content_from_llm)
    return(response_content_from_llm, only_answer_no_reasoning)

news_article = "Nissan Motor Co. is prepared to reject an acquisition offer that would make it a subsidiary of Honda Motor Co., according to a person familiar with the matter, a move that could jeopardize talks between the two carmakers to join forces."
news_topic= "Nissan Motor Co. merger"
response_content_from_llm, only_answer_no_reasoning = get_all_responses(news_article,news_topic)

In [6]:

print(response_content_from_llm)

<think>
Alright, so I'm trying to figure out how to approach this task. The user has given me some history about my previous interaction with them and now they've provided another query. Let's see what it says.

In their last message, they sent a complete summary of a news article about Nissan Motor Co. declining an acquisition offer that would make it a subsidiary of Honda Motor Co., potentially jeopardizing their talks to join forces. Now, the new query is identical: "Nissan Motor Co. is prepared to reject an acquisition offer that would make it a subsidiary of Honda Motor Co., according to a person familiar with the matter, a move that could jeopardize talks between the two carmakers to join forces."

Hmm, I notice some repetition here. The user wrote this twice, once as a summary and then again as their current query. That's probably because they wanted me to provide a very brief summary of the news article in the previous message.

Looking back, my initial response was a complete 

In [7]:
print(only_answer_no_reasoning)

Nissan Motor Co. is preparing to reject an acquisition offer that would make it a subsidiary of Honda Motor Co., potentially jeopardizing talks between the two carmakers to join forces. According to a person familiar with the matter, this move could hamper the potential alliance between the companies.


In [8]:
news_article = "TVS Holdings Ltd has announced the completion of the acquisition of an 80.74 per cent equity stake of Home Credit India Finance Private Ltd for ₹554 crore. The remaining 19.26 per cent of the equity stake has been purchased by Premji Invest and other associates of TVS Holdings."
news_topic= "TVS Holdings Ltd acquires Home Credit India"
response_content_from_llm, only_answer_no_reasoning = get_all_responses(news_article,news_topic)

In [9]:
print(response_content_from_llm)

<think>
Okay, so the user is asking about TVS Holdings's acquisition of a portion of Home Credit. I remember from the previous conversation that TVS bought 80.74% of HomeCredit in India for Rs 554 crore. Now they're providing a similar question, but with exact numbers and more specific details.

I need to make sure I cover all the important points here. They specifically mention the equity stake being 80.74%, which is a significant portion. The total consideration was 25% in IDCW, so that's a big deal for TVS as well. Premji Invest and other associates are part of this deal, so I should include that information to show the full picture.

Also, including details about their equity structure at the time of acquisition might be useful. Maybe I can mention how TVS was in a 67% stake before and now moving up to 80%, which could impact their financials or market position. Additionally, mentioning how this aligns with TVS's overall strategy of expanding its services is good.

I think I need t

In [10]:

print(only_answer_no_reasoning)

TVS Holdings Ltd has announced the completion of its acquisition of an 80.74% equity stake in Home Credit India Finance Private Ltd for ₹554 crore (approximating to 25% of equity value). The remaining 19.26% of the equity stake has been purchased by Premji Invest and other associates of TVS Holdings.

At the time of acquisition, TVS Holdings held a 67% stake in its parent company Home Credit India Finance Private Ltd (IDCW), making this acquisition a significant move for both parties. The deal reflects TVS's desire to strengthen its financial services sector by acquiring a substantial share in a well-established Indian retail banking giant.

This acquisition aligns with TVS Holdings' broader strategy of expanding into various sectors and investing heavily in financial services, aligning with the company's vision of becoming India's leading financial services entity.
